In [41]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import datetime 
import imaplib
import email
from email.header import decode_header
import webbrowser
import os


In [42]:
username = 'jjrekn@gmail.com'
password = 'jakepackerman'
smtp_Object = smtplib.SMTP_SSL('smtp.gmail.com', 465)
smtp_Object.login(username, password)
imap = imaplib.IMAP4_SSL("imap.gmail.com")
imap.login(username, password)
status, messages = imap.select('INBOX')

In [43]:
messages1 = int(messages[0])
input_data = []
for i in range(messages1, messages1-5, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            if subject == 'Basketball Stats':
                From, encoding = decode_header(msg.get("From"))[0]
                if isinstance(From, bytes):
                    From = From.decode(encoding)
                if msg.is_multipart():
                    # iterate over email parts
                    for part in msg.walk():
                        # extract content type of email
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        try:
                            # get the email body
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            # print text/plain emails and skip attachments
                            input_data.append(body)

input_data = input_data[0].split('\r\n')[:-1]
input_data = [x.strip() for x in input_data if x.strip()]

TypeError: can't concat int to bytes

In [40]:
input_data

['Half: First',
 'FGA: 22',
 'FGM: 5',
 'Offensive Rebounds: 7',
 'Assists: 1',
 'Turnovers: 6',
 'Points: 21',
 '2FGM: 4',
 '2FGA: 14',
 '3FGM: 1',
 '3FGA: 8',
 'FTA: 15',
 'FTM: 11']

In [38]:

processed_data = []
for stat in range(len(input_data)):
    new = input_data[stat].split(':')
    processed_data.append(new)



data = {stat[0]:stat[1] for stat in processed_data}
input = pd.DataFrame(processed_data)
input = input.T
new_header = input.iloc[0]

for col in input.columns:
    input[col] = input[col].str.strip()

input = input.rename(columns = new_header).drop(input.index[0])
input

IndexError: list index out of range

In [ ]:
cols = ['FGM','FGA', 'Offensive Rebounds', 'Assists', 'Turnovers', 'Points', '2FGM', '2FGA', '3FGM', '3FGA', 'FTA', 'FTM']

for col in cols:
    input[col] = input[col].astype('float')

if input['Half'].str == 'First':
    for col in cols:
        input[col] = input[col] * 2
    else:
        input[col]

In [ ]:
def offensive_efficiency(row):
   o_e = ((row['FGM'] + row['Assists']) / (row['FGA'] + row['Assists'] + row['Turnovers'] - row['Offensive Rebounds']))
   return o_e

# Efficient Points Scored function 
def efficient_points_scored(row):
   e_p_s = row['Offensive_Efficiency'] * row['Points']
   return e_p_s

# Raw EOP function 
def raw_EOP(row):
    return (.76 * (row['Assists'] + row['Points']) * row['Offensive_Efficiency'])

def get_ppa_two(row):
    total_points = 2 * row['2FGM']
    total_attempts = row['2FGA']
    return total_points / total_attempts

def get_ppa_three(row):
    total_points = 3 * row['3FGM']
    total_attempts = row['3FGA']
    return total_points / total_attempts

def get_total_ppa(row):
    total_points = 2 * row['2FGM'] + 3 * row['3FGM']
    total_attempts = row['FGA']
    return total_points / total_attempts

def team_scoring_poss(row):
    team_attempts = row['FGM']
    inner_part = (1 - (1 - (row['FTM'] / row['FTA']) ** 2))
    return team_attempts + inner_part *  row['FTA'] * 0.4

def team_play_percent(row):
    numerator = row['Scoring_Possesions']
    denom = row['FGA'] + row['FTA'] * 0.4 + row['Turnovers']
    return numerator / denom

def possesions(row):
    fga = row['FGA']
    other_team_def_rebounds = (row['FGA'] - row['FGM']) - row['Offensive Rebounds']
    reb_part = row['Offensive Rebounds']/(row['Offensive Rebounds'] + other_team_def_rebounds)
    fg_part = row['FGA'] - row['FGM']
    return fga - reb_part * fg_part * 1.07 + row['Turnovers'] + row['FTA'] * 0.4

input['Offensive_Efficiency'] = input.apply(offensive_efficiency, axis = 1)
input['Efficient Points Score'] = input.apply(efficient_points_scored, axis = 1)
input['Raw EOP'] = input.apply(raw_EOP, axis = 1)
input['Two Points Per Attempt'] = input.apply(get_ppa_two, axis = 1)
input['Three Points Per Attempt'] = input.apply(get_ppa_three, axis = 1)
input['Total Points Per Attempt'] = input.apply(get_total_ppa, axis = 1)
input['Scoring_Possesions'] = input.apply(team_scoring_poss, axis = 1)
input['Team Play Percent'] = input.apply(team_play_percent, axis = 1)
input['Possesions'] = input.apply(possesions, axis = 1)

input

In [ ]:
import pickle
def game(FieldGoalAttempts, FieldGoalsMade, OffensiveRebounds, Assists, Turnovers, PointsScored, OffensiveEfficiency, EPS, EOP):
   
    return np.array([FieldGoalAttempts, FieldGoalsMade, OffensiveRebounds, Assists, Turnovers, 
    OffensiveEfficiency, 
    EPS, 
    EOP,  PointsScored]).reshape(1,-1)

def predict(game):
    with open(r'C:\Users\Jake\Documents\GitHub\FunAnalyticalStuff\model.bin', 'rb') as f_in:
        model = pickle.load(f_in)
        f_in.close()
    prediction = model.predict(game)
    prediction = pd.DataFrame(prediction)
    prediction = prediction.rename(columns = {0: 'Win or Loss'})
    prediction['Win or Loss'] = prediction['Win or Loss'].apply(lambda x: 'Win' if x == 1 else 'Loss')
    percent_chances = model.predict_proba(game)
    percent_chances = pd.DataFrame(percent_chances)
    percent_chances = percent_chances.rename(columns = {0: 'Loss Percent Chance', 1: 'Win Percent Chance'})
    percent_chances['Loss Percent Chance'] = 100 * percent_chances['Loss Percent Chance']
    percent_chances['Win Percent Chance'] = 100 * percent_chances['Win Percent Chance']
    game_with_percent_chances = [prediction, percent_chances]
    final_prediction = pd.concat(game_with_percent_chances, axis = 1)
    return final_prediction


game = game(input['FGA'], input['FGM'], input['Offensive Rebounds'], input['Assists'], input['Turnovers'], input['Points'], input['Offensive_Efficiency'], input['Efficient Points Score'], input['Raw EOP'])
prediction = predict(game)
input = input.drop(columns = cols)

In [ ]:
concat = pd.concat([prediction, input.reset_index(drop = True)], axis = 1)
concat

In [ ]:
username = 'jjrekn@gmail.com'
password = 'jakepackerman'
#sms_gateway = '4029438850@txt.att.net'
smtp_Object = smtplib.SMTP_SSL('smtp.gmail.com', 465)
smtp_Object.login(username, password)

def send_results(to, concat):
    email = 'jjrekn@gmail.com'
    password = 'jakepackerman'
    #sms_gateway = '4029438850@txt.att.net'
    smtp_Object = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    print('Logging in')
    smtp_Object.login(email, password)
    date = datetime.datetime.now()
    date = date.strftime('%m %d %Y')
    msg = MIMEMultipart()
    subject = 'Predictions and Stats for inputted game'
    msg['From'] = 'jjrekn@gmail.com'
    #msg['To'] = 'morganke3524@gmail.com, jjrekn@gmail.com'
    msg['To'] = 'jjrekn@gmail.com'
    html1 = """\
        <html>
        <head></head>
        <body>
            {0}
        </body>
        </html>
        """.format(concat.to_html())

    print('Attach email')
    part1 = MIMEText(html1, 'html')
    msg.attach(part1)

    print('Send email')
    smtp_Object.sendmail(msg['From'], msg['To'].split(",") , msg.as_string())
    return 'Email sent'

send_results(From, concat)